# Understanding Pandas `DataFrame` operations

In [1]:
import numpy as np
import pandas as pd

## `DataFrame` used for tests

Number of births for each name and year, with an imbalance for genre=F, so we can spot the operations more easily (would be harder to follow what the operations are doing if the F/M data were the same).

In [2]:
births = pd.DataFrame(
    np.array(
        [
            [1970, "Mary", "F", 1],
            [1970, "James", "M", 2],
            [1970, "John", "M", 3],
            [1971, "Mary", "F", 10],
            [1971, "James", "M", 20],
            [1971, "John", "M", 30],
        ]
    ),
    columns=("year", "name", "genre", "births"),
)

births = births.astype({"year": "int", "births": "int"})
births

,year,name,genre,births
0,1970,Mary,F,1
1,1970,James,M,2
2,1970,John,M,3
3,1971,Mary,F,10
4,1971,James,M,20
5,1971,John,M,30


## Group

Calculate number of births by genre in each year.

In [3]:
births_by_genre_year = births.groupby(["year", "genre"]).sum(numeric_only=True)
births_by_genre_year

births
year genre        
1970 F           1
     M           5
1971 F          10
     M          50

Although not wrong, it's not the easiest way to visualize the numbers. Move `genre` to columns with `unstack()`:

In [4]:
births_by_genre_year = births_by_genre_year.unstack()
births_by_genre_year

births    
genre      F   M
year            
1970       1   5
1971      10  50

Pivot table version - do it all in one shot.

In [5]:
births.pivot_table("births", index="year", columns="genre", aggfunc="sum")

genre,F,M
year,,
1970,1,5
1971,10,50


## Slicing

Picking the `DataFrame` apart by addressing specific pieces of it.

Number of births for a specific year and genre.

In [6]:
births_by_genre_year.loc[1970]["births"]["M"]

5

Number of births for a specific genre, all years.

In [7]:
births_by_genre_year.loc[:]["births"]["M"]

year
1970     5
1971    50
Name: M, dtype: int64

## Aggregations - transform and augment data

**VERY IMPORTANT CONCEPT**: the axis specifies the dimension that will be collapsed (i.e. will be operated on and consolidated).

If we ask to perform an opeartion on rows, the rows are collapsed and aggreated by the specified operation.

### By rows: no axis, axis=0, axis='rows'

Collapse the rows and leave columns in place, i.e. aggregate (sums) row values.

In [8]:
# These are equivalent
births_by_genre_year.sum().unstack()
births_by_genre_year.sum(axis=0).unstack()
births_by_genre_year.sum(axis="rows").unstack()

genre,F,M
births,11,55


### By columns: axis=1, axis='columns'

Collapse the columns and leave the rows in place, i.e. aggregate (sums) colum values.

In [9]:
# These are equivalent
births_by_genre_year.sum(axis=1)
births_by_genre_year.sum(axis="columns")

year
1970     6
1971    60
dtype: int64

### `groupby`: split, apply, combine

#### Calculating the percentage of M/F in each year

How it works:

* **Split**: Since we want the percentage by year and by genre, we need to split using both year and genre.
* **Apply**: Once the dataset is split, we can apply the calculations we need to that slice.
* **Combine**: Pandas reassembles the results of the _apply_ step into one `DataFrame`.

In the code below, the `pct_genre_year(group)` is the _apply_ step. The `print(...)` statements are not needed for the calculations. They are there to help us visualize how the data set was split.

In [10]:
def pct_genre_year(group):
    print(
        "\nThis is the split for {},{}".format(group.year.iloc[0], group.genre.iloc[0])
    )
    print(group)

    group["pct_genre_year"] = group.births / group.births.sum() * 100
    return group


births_pct_year_genre = births.groupby(["year", "genre"], group_keys=False).apply(
    pct_genre_year
)
births_pct_year_genre


This is the split for 1970,F
   year  name genre  births
0  1970  Mary     F       1

This is the split for 1970,M
   year   name genre  births
1  1970  James     M       2
2  1970   John     M       3

This is the split for 1971,F
   year  name genre  births
3  1971  Mary     F      10

This is the split for 1971,M
   year   name genre  births
4  1971  James     M      20
5  1971   John     M      30


,year,name,genre,births,pct_genre_year
0,1970,Mary,F,1,100.0
1,1970,James,M,2,40.0
2,1970,John,M,3,60.0
3,1971,Mary,F,10,100.0
4,1971,James,M,20,40.0
5,1971,John,M,30,60.0


Visually:

![Split, apply, combine example](images/split_apply_combine_births.png)

#### Calculating number of births per year

We do not need to return the same number of rows when grouping a dataset.

In this example we consolidate the M and F columns to get an overall count of births per year, independent of genre.

In [11]:
def births_per_year(group):
    return group["births"].sum()


births.groupby(["year"]).apply(births_per_year)

year
1970     6
1971    60
dtype: int64

When the `apply` function is simple, we can use a lambda expression:

In [12]:
births.groupby(["year"]).apply(lambda x: x["births"].sum())

year
1970     6
1971    60
dtype: int64

We can clean up the syntax a bit:

* Remove the array from `groupby`, since we are grouping by one column only
* Use the column name `births` as a variable name

In [13]:
births.groupby("year").apply(lambda x: x.births.sum())

year
1970     6
1971    60
dtype: int64

Finally, `pivot_table` may be a clearer alternative when consolidating information.

In [14]:
births.pivot_table("births", index="year", aggfunc="sum")

,births
year,
1970,6
1971,60


The `pivot_table` above works for this case because we ommited the `columns` parameter. If we add it back in, we break the results down further (as we used above, in the [group example](#Group)).

In [15]:
births.pivot_table("births", index="year", columns="genre", aggfunc="sum")

genre,F,M
year,,
1970,1,5
1971,10,50


**************
TODO: differences between groupby and pivot_table, e.g. Series vs. DataFrame. Other guidelines?
**************

## Querying - asking questions and getting answers

#### How many boys were born overall (in all years)?

In [16]:
# Save as a filter to use it later
boys = births.genre == "M"
births[boys].births.sum()

55

#### How many girls were born overall (in all years)?

In [17]:
# Save as a filter to use it later
girls = births.genre == "F"
births[girls].births.sum()

11

In [18]:
# Since there are only two options for genre, the ~ operator also works
girls = ~boys
births[girls].births.sum()

11

#### How many boys born in 1970?

In [19]:
births[boys & (births.year == 1970)].births.sum()

5

In [20]:
# Using query()
births.query('genre == "M" & year == 1970').births.sum()

5

**************
TODO: when to use query vs. regular filtering
**************

#### What names were used more than ten times in one year?

In [21]:
births[births.births > 10]

,year,name,genre,births
4,1971,James,M,20
5,1971,John,M,30


#### What is the number boys and girls born in each year?

In [22]:
births.groupby(["year", "genre"]).sum(numeric_only=True)

births
year genre        
1970 F           1
     M           5
1971 F          10
     M          50

In [23]:
# Using pivot_table()
births.pivot_table("births", index="year", columns="genre", aggfunc="sum")

genre,F,M
year,,
1970,1,5
1971,10,50


In [24]:
# Make the groupby() look like the pivot_table()
births.groupby(["year", "genre"]).sum(numeric_only=True).unstack()

births    
genre      F   M
year            
1970       1   5
1971      10  50

In [25]:
# Make the pivot_table() look like the first groupby()
births.pivot_table("births", index=["year", "genre"], aggfunc="sum")

births
year genre        
1970 F           1
     M           5
1971 F          10
     M          50

#### What is the year with the highest number of births?

In [26]:
# groupby() sorts ascending, so highest value is the last one
births.groupby("year", sort=True).births.sum().iloc[-1:]

year
1971    60
Name: births, dtype: int64

In [27]:
# if we want the year only
births.groupby("year", sort=True).births.sum().iloc[-1:].index[0]

1971

In [28]:
# With pivot_table()
births.pivot_table("births", index="year", aggfunc="sum")[-1:]

,births
year,
1971,60


In [29]:
# Difference between groupby() and pivot_table()
print(type(births.groupby("year", sort=True).births.sum()))
print(type(births.pivot_table("births", index="year", aggfunc="sum")))

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>


#### What is the year with the lowest number of girl births?

In [30]:
# Reuse the filter we created many cells ago
# groupby() sorts ascending, so lowest value is the first one
births[girls].groupby("year", sort=True).births.sum().iloc[:1]

year
1970    1
Name: births, dtype: int64

## A word about `value_counts`

If we are interested only in the count or proportion of categorical columns, we can use `values_counts`:

In [31]:
births["genre"].value_counts()

genre
M    4
F    2
Name: count, dtype: int64

In [32]:
births["genre"].value_counts(normalize=True)

genre
M    0.666667
F    0.333333
Name: proportion, dtype: float64